In [22]:
### DONE ::  1,2,3,5,6,7,10,11,13,15,19

In [24]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()

buy = [[solver.NumVar(0, 1000, f'buy[{i}][{j}]') for j in range(5)] for i in range(6)]
used = [[solver.NumVar(0, solver.infinity(), f'used[{i}][{j}]') for j in range(5)] for i in range(6)]
store = [[solver.NumVar(0, solver.infinity(), f'store[{i}][{j}]') for j in range(5)] for i in range(6)]

for i in range(6):
#### given need not exceed 200 veg and 250 non veg 
    solver.Add(used[i][0]+ used[i][1] <= 200)
    solver.Add(used[i][3]+used[i][2]+ used[i][4]<= 250)

    for j in range(5):
        if i == 0:
            ### intially has 500 and ending also 500 
            solver.Add(buy[i][j] - used[i][j] - store[i][j] == -500)
        elif i == 5:
            solver.Add(buy[i][j] - used[i][j] - store[i][j]+store[i-1][j] == 500)
        else:
            solver.Add(buy[i][j] - used[i][j] - store[i][j]+store[i-1][j] == 0)

hardmax = [2.8, 0.1, -4.0, -1.8, -1.0]
hardmin = [5.8, 3.1, -1.0, 1.2, 2.0]

for i in range(6):
    temp1 = sum(hardmax[j] * used[i][j] for j in range(5))
    temp2 = sum(hardmin[j] * used[i][j] for j in range(5))
    solver.Add(temp1 <= 0.0)
    solver.Add(temp2 >= 0.0)

prices = [
    [110, 120, 130, 110, 115],  
    [130, 130, 110, 90, 115],   
    [110, 140, 130, 100, 95],   
    [120, 110, 120, 120, 125],  
    [100, 120, 150, 110, 105],  
    [90, 100, 140, 80, 135]     
]

result = 0
for i in range(6):
    for j in range(5):
        result += -prices[i][j] * buy[i][j] + 150 * used[i][j] -5*store[i][j]
solver.Maximize(result)
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value()}')

    months = ["January", "February", "March", "April", "May", "June"]
    products = ["veg 1", "veg  2", "oil 1", "oil 2", "oil 3"]

    header = "Month       " + "  ".join([f"{prod:<10}" for prod in products])
    print(header)


    for i in range(6):
        item_row = f"{months[i]:<12}" + "  ".join([f"{buy[i][j].solution_value():<10.2f}" for j in range(5)])
        sell_row = f"{'':<12}" + "  ".join([f"{used[i][j].solution_value():<10.2f}" for j in range(5)])
        store_row = f"{'':<12}" + "  ".join([f"{store[i][j].solution_value():<10.2f}" for j in range(5)])
        
        print(item_row)
        print(sell_row)
        print(store_row)
        print("-" * len(header))
else:
    print('The problem does not have an optimal solution.')


Solution:
Objective value = 120342.59259259258
Month       veg 1       veg  2      oil 1       oil 2       oil 3     
January     0.00        0.00        0.00        0.00        0.00      
            0.00        200.00      0.00        250.00      0.00      
            500.00      300.00      500.00      250.00      500.00    
----------------------------------------------------------------------
February    0.00        0.00        0.00        750.00      0.00      
            159.26      40.74       0.00        250.00      0.00      
            340.74      259.26      500.00      750.00      500.00    
----------------------------------------------------------------------
March       0.00        -0.00       0.00        -0.00       0.00      
            159.26      40.74       0.00        250.00      0.00      
            181.48      218.52      500.00      500.00      500.00    
----------------------------------------------------------------------
April       0.00        -0.00 

In [25]:
####  12.2

In [26]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()

buy = [[solver.NumVar(0, solver.infinity(), f'buy[{i}][{j}]') for j in range(5)] for i in range(6)]
used = [[solver.NumVar(0, solver.infinity(), f'used[{i}][{j}]') for j in range(5)] for i in range(6)]
store = [[solver.NumVar(0, solver.infinity(), f'store[{i}][{j}]') for j in range(5)] for i in range(6)]
usedby =[[solver.BoolVar(f'oil {j} used in month {i}') for j in range(5)] for i in range(6)]
for i in range(6):
    ##max three oils 
    solver.Add(sum(usedby[i][j] for j in range(5))<=3)
    ## if either oil 1 or oil 2 then oil 5 
    solver.Add(usedby[i][0] + usedby[i][1] <= 2*usedby[i][4])
    for j in range(5):
        ##  if oil used min 20 is used
        solver.Add(used[i][j] >= 20 * usedby[i][j])
        if j==0 or  j==1:
            solver.Add(used[i][j]<=200*usedby[i][j])
        else :
            solver.Add(used[i][j]<=250*usedby[i][j])
    solver.Add(used[i][0]+ used[i][1] <= 200)
    solver.Add(used[i][3]+used[i][2]+ used[i][4]<= 250)

    for j in range(5):
        if i == 0:
            solver.Add(buy[i][j] - used[i][j] - store[i][j] == -500)
        elif i == 5:
            solver.Add(buy[i][j] - used[i][j] +store[i-1][j] == 500)
        else:
            solver.Add(buy[i][j] - used[i][j] - store[i][j]+store[i-1][j] == 0)
        

hardmax = [2.8, 0.1, -4.0, -1.8, -1.0]
hardmin = [5.8, 3.1, -1.0, 1.2, 2.0]

for i in range(6):
    temp1 = sum(hardmax[j] * used[i][j] for j in range(5))
    temp2 = sum(hardmin[j] * used[i][j] for j in range(5))
    solver.Add(temp1 <= 0.00)
    solver.Add(temp2 >= 0.00)

prices = [
    [110, 120, 130, 110, 115],  
    [130, 130, 110, 90, 115],   
    [110, 140, 130, 100, 95],   
    [120, 110, 120, 120, 125],  
    [100, 120, 150, 110, 105],  
    [90, 100, 140, 80, 135]     
]

result = 0
for i in range(6):
    for j in range(5):
        if i==5:
            solver.Add(store[i][j]==500)
        result += -prices[i][j] * buy[i][j] + 150 * used[i][j] -5*store[i][j]
solver.Mapathimize(result)
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value()}')

    months = ["January", "February", "March", "April", "May", "June"]
    products = ["veg 1", "veg  2", "oil 1", "oil 2", "oil 3"]

    header = "Month       " + "  ".join([f"{prod:<10}" for prod in products])
    print(header)


    for i in range(6):
        item_row = f"{months[i]:<12}" + "  ".join([f"{buy[i][j].solution_value():<10.2f}" for j in range(5)])
        sell_row = f"{'':<12}" + "  ".join([f"{used[i][j].solution_value():<10.2f}" for j in range(5)])
        store_row = f"{'':<12}" + "  ".join([f"{store[i][j].solution_value():<10.2f}" for j in range(5)])
        
        print(item_row)
        print(sell_row)
        print(store_row)
        print("-" * len(header))
else:
    print('The problem does not have an optimal solution.')

AttributeError: 'Solver' object has no attribute 'Mapathimize'

In [ ]:
### 12.3 

In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    epathit()

items=[[solver.NumVar(0,solver.infinity(),f'items[{i}][{j}]') for j in range(7)] for i in range(6)]
sell=[[solver.NumVar(0,solver.infinity(),f'sell[{i}][{j}]') for j in range(7)] for i in range(6)]
store=[[solver.NumVar(0,100,f'store[{i}][{j}]') for j in range(7)] for i in range(6)]
42311
machines=[[3,2,3,1,1],[4,2,1,1,1],[4,2,3,0,1],[4,1,3,1,1],[3,1,3,1,1],[4,2,2,1,0]]
hours= [
    [0.5, 0.7, 0.0, 0.0, 0.3, 0.2, 0.5], 
    [0.1, 0.2, 0.0, 0.3, 0.0, 0.6, 0.0],  
    [0.2, 0.0, 0.8, 0.0, 0.0, 0.0, 0.6],  
    [0.05, 0.03, 0.0, 0.07, 0.1, 0.0, 0.08],  
    [0.0, 0.0, 0.01, 0.0, 0.05, 0.0, 0.05]   
]
limit = [
    [500, 1000, 300, 300, 800, 200, 100],   
    [600, 500, 200, 0, 400, 300, 150],      
    [300, 600, 0, 0, 500, 400, 100],        
    [200, 300, 400, 500, 200, 0, 100],      
    [0, 100, 500, 100, 1000, 300, 0],       
    [500, 500, 100, 300, 1100, 500, 60]     
]
for i in range(6):
    for j in range(7):
        solver.Add(items[i][j] <= limit[i][j])
    for j in range(5):
        solver.Add(sum(hours[j][k]*items[i][k] for k in range(7))<=machines[i][j]*24*16)
for i in range(6):
    for j in range(7):
        if i==0:
            solver.Add(items[i][j]==sell[i][j]+store[i][j])
        else:
            solver.Add(items[i][j]+store[i-1][j]==sell[i][j]+store[i][j])
for j in range(7):
    solver.Add(store[5][j]==50)
result=0
cost=[10,6,8,4,11,9,3]
for i in range(6):
    for j in range(7):
        result+=cost[j]*sell[i][j]-0.5*store[i][j]
solver.Mapathimize(result)
solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value()}')

    months = ["January", "February", "March", "April", "May", "June"]
    products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6", "Product 7"]

    header = "Month       " + "  ".join([f"{prod:<10}" for prod in products])
    print(header)


    for i in range(6):
        item_row = f"{months[i]:<12}" + "  ".join([f"{items[i][j].solution_value():<10.2f}" for j in range(7)])
        sell_row = f"{'':<12}" + "  ".join([f"{sell[i][j].solution_value():<10.2f}" for j in range(7)])
        store_row = f"{'':<12}" + "  ".join([f"{store[i][j].solution_value():<10.2f}" for j in range(7)])
        
        print(item_row)
        print(sell_row)
        print(store_row)
        print("-" * len(header))
else:
    print('The problem does not have an optimal solution.')


Solution:
Objective value = 87356.42857142857
Month       Product 1   Product 2   Product 3   Product 4   Product 5   Product 6   Product 7 
January     500.00      888.57      300.00      300.00      800.00      200.00      0.00      
            500.00      888.57      300.00      300.00      800.00      200.00      0.00      
            0.00        0.00        0.00        0.00        0.00        0.00        0.00      
----------------------------------------------------------------------------------------------
February    600.00      500.00      200.00      0.00        400.00      300.00      150.00    
            600.00      500.00      200.00      0.00        400.00      300.00      150.00    
            0.00        0.00        0.00        0.00        0.00        0.00        0.00      
----------------------------------------------------------------------------------------------
March       0.00        0.00        0.00        0.00        0.00        400.00      0.00      
    

In [ ]:
###    12.5

In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    epathit()


num_years = 3
num_variables = 3 * num_years  
## total
tSK = [solver.NumVar(0, solver.infinity(), f'tSK[{i}]') for i in range(4)] 
tSS = [solver.NumVar(0, solver.infinity(), f'tSS[{i}]') for i in range(4)]
tUS = [solver.NumVar(0, solver.infinity(), f'tUS[{i}]') for i in range(4)]
## recuited
uSK = [solver.NumVar(0, 500, f'uSK[{i}]') for i in range(num_years)]
uSS = [solver.NumVar(0, 800, f'uSS[{i}]') for i in range(num_years)]
uUS = [solver.NumVar(0, 500, f'uUS[{i}]') for i in range(num_years)]
## retrained
vUSSS = [solver.NumVar(0, 200, f'vUSSS[{i}]') for i in range(num_years)]  ## unskill to semi skill
vSSSK = [solver.NumVar(0, solver.infinity(), f'vSSSK[{i}]') for i in range(num_years)]  ## semi to skill
## down
vSKSS = [solver.NumVar(0, solver.infinity(), f'vSKSS[{i}]') for i in range(num_years)]  ## skill to semi
uSKUS = [solver.NumVar(0, solver.infinity(), f'uSKUS[{i}]') for i in range(num_years)]   ## skill to unskill
vSSUS = [solver.NumVar(0, solver.infinity(), f'vSSUS[{i}]') for i in range(num_years)]   ## semi to uns kill 
## redundacy
wSK = [solver.NumVar(0, solver.infinity(), f'wSK[{i}]') for i in range(num_years)]
wSS = [solver.NumVar(0, solver.infinity(), f'wSS[{i}]') for i in range(num_years)]
wUS = [solver.NumVar(0, solver.infinity(), f'wUS[{i}]') for i in range(num_years)]
## short time
pathSK = [solver.NumVar(0,50, f'pathSK[{i}]') for i in range(num_years)]
pathSS = [solver.NumVar(0, 50, f'xSS[{i}]') for i in range(num_years)]
xUS = [solver.NumVar(0, 50, f'xUS[{i}]') for i in range(num_years)]
#3 over man 
ySK = [solver.NumVar(0, solver.infinity(), f'ySK[{i}]') for i in range(num_years)]
ySS = [solver.NumVar(0, solver.infinity(), f'ySS[{i}]') for i in range(num_years)]
yUS = [solver.NumVar(0, solver.infinity(), f'yUS[{i}]') for i in range(num_years)]

solver.Add(tSK[0]==1000)
solver.Add(tSS[0]==1500)
solver.Add(tUS[0]==2000)

for i in range(1,4):
    skill=[1000,1500,2000]
    semi=[1400,2000,2500]
    un=[1000,500,0]
    solver.Add(tSK[i]-ySK[i-1]-0.5*xSK[i-1]==skill[i-1])
    solver.Add(tSS[i]-ySS[i-1]-0.5*xSS[i-1]==semi[i-1])
    solver.Add(tUS[i]-yUS[i-1]-0.5*xUS[i-1]==un[i-1])
for i in range(3):
    solver.Add(vSSSK[i]-0.25*tSK[i+1]<=0)
    solver.Add(ySK[i]+ySS[i]+yUS[i]<=150)

for i in range(3):

    solver.Add(tSK[i+1] == 0.95 * tSK[i] + 0.9 * uSK[i] + vSSSK[i] - vSKSS[i] - uSKUS[i] - wSK[i])
    solver.Add(tSS[i+1] == 0.95 * tSS[i] + 0.8 * uSS[i] + vUSSS[i] - vSSSK[i] + 0.5 * vSKSS[i] - wSS[i] - vSSUS[i])
    solver.Add(tUS[i+1] == 0.9 * tUS[i] + 0.75 * uUS[i] - vUSSS[i] + 0.5 * vSKSS[i] + 0.5 * vSSUS[i] - wUS[i])


result=0
for i in range(3):
    result+=wSK[i]+wSS[i]+wUS[i]

solver.Minimize(result)
status=solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value()}')

    months = ["Year 1", "Year 2", "Year 3"]
    variables = ["tSK", "tSS", "tUS", "uSK", "uSS", "uUS", "vUSSS", "vSSSK", "vSKSS", "uSKUS", "vSSUS", "wSK", "wSS", "wUS", "xSK", "xSS", "xUS", "ySK", "ySS", "yUS"]

    # Print header
    header = "Variable".ljust(10) + " ".join([f"{month:<10}" for month in months])
    print(header)

    # Print variable values
    for var in variables:
        var_name = var.ljust(10)
        var_values = " ".join([f"{eval(var)[i].solution_value():<10.2f}" for i in range(num_years)])
        print(var_name + var_values)

else:
    print('The problem does not have an optimal solution.')


Solution:
Objective value = 840.0
Variable  Year 1     Year 2     Year 3    
tSK       1000.00    1025.00    1525.00   
tSS       1500.00    1425.00    2025.00   
tUS       2000.00    1175.00    675.00    
uSK       500.00     500.00     500.00    
uSS       70.31      715.62     596.88    
uUS       0.00       0.00       0.00      
vUSSS     200.00     200.00     200.00    
vSSSK     256.25     101.25     101.25    
vSKSS     0.00       0.00       0.00      
uSKUS     631.25     -0.00      0.00      
vSSUS     0.00       0.00       -0.00     
wSK       0.00       0.00       0.00      
wSS       -0.00      0.00       0.00      
wUS       425.00     182.50     232.50    
xSK       50.00      50.00      0.00      
xSS       50.00      50.00      0.00      
xUS       50.00      50.00      50.00     
ySK       0.00       0.00       0.00      
ySS       0.00       0.00       0.00      
yUS       150.00     150.00     150.00    


In [ ]:
result=0
for i in range(3):
    result+=400*vUSSS[i]+500*vSSSK[i]+200*wUS[i]+500*(wSS[i]+wSK[i])+400*xSK[i]+400*xSS[i]+500*xUS[i]+3000*ySK[i]+2000*ySS[i]+1500*yUS[i]

solver.Minimize(result)
status=solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value()}')

    months = ["Year 1", "Year 2", "Year 3"]
    variables = ["tSK", "tSS", "tUS", "uSK", "uSS", "uUS", "vUSSS", "vSSSK", "vSKSS", "uSKUS", "vSSUS", "wSK", "wSS", "wUS", "xSK", "xSS", "xUS", "ySK", "ySS", "yUS"]

    header = "Variable".ljust(10) + " ".join([f"{month:<10}" for month in months])
    print(header)

    for var in variables:
        var_name = var.ljust(10)
        var_values = " ".join([f"{eval(var)[i].solution_value():<10.2f}" for i in range(num_years)])
        print(var_name + var_values)

else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 488000.0
Variable  Year 1     Year 2     Year 3    
tSK       1000.00    1000.00    1500.00   
tSS       1500.00    1400.00    2000.00   
tUS       2000.00    1000.00    500.00    
uSK       500.00     500.00     500.00    
uSS       0.00       800.00     800.00    
uUS       0.00       0.00       0.00      
vUSSS     0.00       130.00     85.00     
vSSSK     0.00       100.00     125.00    
vSKSS     0.00       0.00       0.00      
uSKUS     400.00     0.00       0.00      
vSSUS     25.00      0.00       0.00      
wSK       0.00       0.00       0.00      
wSS       -0.00      0.00       -0.00     
wUS       812.50     270.00     365.00    
xSK       0.00       0.00       0.00      
xSS       0.00       0.00       0.00      
xUS       0.00       0.00       0.00      
ySK       0.00       0.00       0.00      
ySS       0.00       0.00       0.00      
yUS       0.00       0.00       0.00      


In [ ]:
###12.6

In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('CBC')
if not solver:
    print('Solver not created.')
    exit()

# Define variables
c_1 = solver.NumVar(0, solver.infinity(), f'c_1')
c_2 = solver.NumVar(0, solver.infinity(), f'c_2')

light_nap = solver.NumVar(0, solver.infinity(), f'light_nap')
med_nap = solver.NumVar(0, solver.infinity(), f'med_nap')
high_nap = solver.NumVar(0, solver.infinity(), f'high_nap')
light_oil = solver.NumVar(0, solver.infinity(), f'light_oil')
res = solver.NumVar(0, solver.infinity(), f'res')
high_oil = solver.NumVar(0, solver.infinity(), f'high_oil')

light_nap_used_reformedgasoline = solver.NumVar(0, solver.infinity(), f'light_nap_used_reformedgasoline')
med_nap_used_reformedgasoline = solver.NumVar(0, solver.infinity(), f'med_nap_used_reformedgasoline')
high_nap_used_reformedgasoline = solver.NumVar(0, solver.infinity(), f'high_nap_used_reformedgasoline')

reformedgasoline = solver.NumVar(0, solver.infinity(), f'reformedgasoline')

light_oil_used_cracked = solver.NumVar(0, solver.infinity(), f'light_oil_used_cracked')
high_oil_used_cracked = solver.NumVar(0, solver.infinity(), f'high_oil_used_cracked')

crackedoil = solver.NumVar(0, solver.infinity(), f'crackedoil')
crackedgas = solver.NumVar(0, solver.infinity(), f'crackedgas')


lubeoil = solver.NumVar(0, solver.infinity(), f'lubeoil')
res_used_lubeoil = solver.NumVar(0, solver.infinity(), f'res_used_lubeoil')


lightnap_premium = solver.NumVar(0, solver.infinity(), f'lightnap_premium')
mednap_premium = solver.NumVar(0, solver.infinity(), f'mednap_premium')
highnap_premium = solver.NumVar(0, solver.infinity(), f'highnap_premium')
reformedgas_premium = solver.NumVar(0, solver.infinity(), f'reformedgas_premium')
crakedgas_premium = solver.NumVar(0, solver.infinity(), f'crakedgas_premium')

lightnap_regular = solver.NumVar(0, solver.infinity(), f'lightnap_regular')
mednap_regular = solver.NumVar(0, solver.infinity(), f'mednap_regular')
highnap_regular = solver.NumVar(0, solver.infinity(), f'highnap_regular')
reformedgas_regular = solver.NumVar(0, solver.infinity(), f'reformedgas_regular')
crakedgas_regular = solver.NumVar(0, solver.infinity(), f'crakedgas_regular')

premium = solver.NumVar(0, solver.infinity(), f'premium')
regular = solver.NumVar(0, solver.infinity(), f'regular')


lightoil_usedjet = solver.NumVar(0, solver.infinity(), f'lightoil_usedjet')
highoil_usedjet = solver.NumVar(0, solver.infinity(), f'highoil_usedjet')
crackoil_usedjet = solver.NumVar(0, solver.infinity(), f'crackoil_usedjet')
res_usedjet = solver.NumVar(0, solver.infinity(), f'res_usedjet')
jet = solver.NumVar(0, solver.infinity(), f'jet')

lightoil_usedfuel = solver.NumVar(0, solver.infinity(), f'lightoil_usedfuel')
highoil_usedfuel = solver.NumVar(0, solver.infinity(), f'highoil_usedfuel')
crackoil_usedfuel = solver.NumVar(0, solver.infinity(), f'crackoil_usedfuel')
res_usedfuel = solver.NumVar(0, solver.infinity(), f'res_usedfuel')
fuel = solver.NumVar(0, solver.infinity(), f'fuel')

solver.Add(light_nap == 0.1 * c_1 + 0.15 * c_2)
solver.Add(med_nap == 0.2 * c_1 + 0.25 * c_2)
solver.Add(high_nap == 0.2 * c_1 + 0.18 * c_2)
solver.Add(light_oil == 0.12 * c_1 + 0.08 * c_2)
solver.Add(high_oil == 0.2 * c_1 + 0.19 * c_2)
solver.Add(res == 0.13 * c_1 + 0.12 * c_2)
solver.Add(reformedgasoline == 0.6 * light_nap_used_reformedgasoline +
           0.52 * med_nap_used_reformedgasoline +
           0.45 * high_nap_used_reformedgasoline)

solver.Add(crackedoil == 0.68 * light_oil_used_cracked +
           0.75 * high_oil_used_cracked)

solver.Add(crackedgas == 0.28 * light_oil_used_cracked +
           0.2 * high_oil_used_cracked)

solver.Add(lubeoil == 0.5 * res_used_lubeoil)

solver.Add(90*lightnap_premium+80*mednap_premium+70*highnap_premium+115*reformedgas_premium+105*crakedgas_premium-94*premium>=0)
solver.Add(90*lightnap_regular+80*mednap_regular+70*highnap_regular+105*crakedgas_regular+115*reformedgas_regular-84*regular>=0)
solver.Add(jet-lightoil_usedjet-0.6*highoil_usedjet-1.5*crackoil_usedjet-0.05*res_usedjet>=0)

solver.Add(fuel*10==18*lightoil_usedfuel)
solver.Add(fuel*4==18*crackoil_usedfuel)
solver.Add(fuel*3==18*highoil_usedfuel)
solver.Add(fuel*1==18*res_usedfuel)
###solver.Add(fuel==lightoil_usedfuel+crackoil_usedfuel+highoil_usedfuel+res_usedfuel)

solver.Add(c_1<=20000)
solver.Add(c_2<=30000)
solver.Add(c_1+c_2<=45000)
solver.Add(jet==lightoil_usedjet+highoil_usedjet+crackoil_usedjet+res_usedjet)
solver.Add(premium==lightnap_premium+mednap_premium+highnap_premium+reformedgas_premium+crakedgas_premium)
solver.Add(regular==lightnap_regular+mednap_regular+highnap_regular+reformedgas_regular+crakedgas_regular)

solver.Add(light_nap_used_reformedgasoline + med_nap_used_reformedgasoline +
           high_nap_used_reformedgasoline <= 10000)

solver.Add(light_oil_used_cracked + high_oil_used_cracked <= 8000)
solver.Add(500<=lubeoil)
solver.Add(lubeoil<=1000)
solver.Add(premium>=0.4 * regular)

solver.Add(light_nap==light_nap_used_reformedgasoline+lightnap_premium+lightnap_regular)
solver.Add(med_nap==med_nap_used_reformedgasoline+mednap_premium+mednap_regular)
##solver.Add(high_nap==high_nap_used_reformedgasoline+highnap_regular+highnap_regular)
solver.Add(high_nap == high_nap_used_reformedgasoline + highnap_premium + highnap_regular)

solver.Add(light_oil==light_oil_used_cracked+lightoil_usedfuel+lightoil_usedjet)
solver.Add(high_oil==highoil_usedfuel+highoil_usedjet+high_oil_used_cracked)
solver.Add(res==res_used_lubeoil+res_usedfuel+res_usedjet)
solver.Add(crackedoil==crackoil_usedfuel+crackoil_usedjet)
solver.Add(crackedgas==crakedgas_premium+crakedgas_regular)
# Objective function
result = 7 * premium + 6 * regular + 4 * jet + 3.5 * fuel + 1.5 * lubeoil
solver.Maximize(result)

# Solve the problem
status = solver.Solve()

# Print solution
if  status == pywraplp.Solver.OPTIMAL :
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value()}')

    print(f'Premium: {premium.solution_value()}')
    print(f'Regular: {regular.solution_value()}')
    print(f'Jet Fuel: {jet.solution_value()}')
    print(f'Fuel Oil: {fuel.solution_value()}')
    print(f'Lube Oil: {lubeoil.solution_value()}')

else:
    print('The problem does not have an optimal solution.')


The problem does not have an optimal solution.


In [ ]:
### 12.7 

In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()

operate = [[solver.BoolVar(f'mine{i} is operated in year{j}') for i in range(4)]for j in range(5)]
open = [[solver.BoolVar(f'mine{i} is open in year{j}') for i in range(4)]for j in range(5)]
output=[[solver.NumVar(0,solver.infinity(),'output') for i in range(4)]for j in range(5)]

## atmost 3 mine sin a year
for j in range(5):
    solver.Add(solver.Sum(operate[j][i] for i in range(4)) <= 3)
## if it is closed one year then it is closed further also 
for i in range(4):
    for j in range(4):
        solver.Add(open[j+1][i]-open[j][i]<=0)
        
capacity=[2000000,2500000,1300000,3000000]
mine_quality=[1.0,0.7,1.5,0.5]
year_quality=[0.9,0.8,1.2,0.6,1.0]
## upper limit of capacities 
for j in range(5):
    for i in range(4):
        solver.Add(output[j][i]<=capacity[i]*operate[j][i])
        solver.Add(operate[j][i]-open[j][i]<=0)
for j in range(5):
    tot=0
    for i in range(4):
        tot+=(year_quality[j]*output[j][i]-mine_quality[i]*output[j][i])
    solver.Add(tot==0)
royal=[5,4,4,5]
result= 0
for j in range(5):
    for i in range(4):
        result+=(10*output[j][i]-royal[i]*open[j][i])


solver.Maximize(result)
##solver.SetTimeLimit(10000)
# Solve the problem
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value()}')
    for j in range(5):
        for i in range(4):
            print(f'Mine {i+1} in year {j+1} - Operate: {operate[j][i].solution_value()}, Open: {open[j][i].solution_value()}, Output: {output[j][i].solution_value()/(10**6)}')
else:
    print('The solver did not find an optimal solution.')

# Debugging: Print constraint violations if any
for constraint in solver.constraints():
    if constraint.Lb() > constraint.Ub():
        print(f'Constraint violated: {constraint}')


Solution:
Objective value = 260916581.66666666
Mine 1 in year 1 - Operate: 1.0, Open: 1.0, Output: 1.9999999999999998
Mine 2 in year 1 - Operate: 0.0, Open: 1.0, Output: 0.0
Mine 3 in year 1 - Operate: 1.0, Open: 1.0, Output: 1.3000000000000003
Mine 4 in year 1 - Operate: 1.0, Open: 1.0, Output: 2.45
Mine 1 in year 2 - Operate: 0.0, Open: 1.0, Output: 0.0
Mine 2 in year 2 - Operate: 1.0, Open: 1.0, Output: 2.5
Mine 3 in year 2 - Operate: 1.0, Open: 1.0, Output: 1.3
Mine 4 in year 2 - Operate: 1.0, Open: 1.0, Output: 2.1999999999999984
Mine 1 in year 3 - Operate: 1.0, Open: 1.0, Output: 1.9500000000000008
Mine 2 in year 3 - Operate: 0.0, Open: 1.0, Output: 0.0
Mine 3 in year 3 - Operate: 1.0, Open: 1.0, Output: 1.3
Mine 4 in year 3 - Operate: 0.0, Open: 1.0, Output: 0.0
Mine 1 in year 4 - Operate: 1.0, Open: 1.0, Output: 0.12499999999999986
Mine 2 in year 4 - Operate: 1.0, Open: 1.0, Output: 2.5
Mine 3 in year 4 - Operate: 0.0, Open: 1.0, Output: 0.0
Mine 4 in year 4 - Operate: 1.0, Ope

In [ ]:
### 12.10

In [34]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()
open = [[solver.BoolVar(f'city_{i}_has_dept_{j}') for j in range(5)] for i in range(3)]
path = [[[[
    solver.BoolVar(f'city_{i}_has_dept_{j}_and_city_{k}_has_dept_{l}')
    for l in range(5)]
    for k in range(3)]
    for j in range(5)]
    for i in range(3)]

for i in range(3):
    solver.Add(solver.Sum(open[i][j] for j in range(5)) <= 3)

for j in range(5):
    solver.Add(solver.Sum(open[i][j] for i in range(3)) == 1)

for i in range(3):
    for j in range(5):
        for k in range(3):
            for l in range(5):
                solver.Add(path[i][j][k][l] <= open[i][j])
                solver.Add(path[i][j][k][l] <= open[k][l])
                solver.Add(path[i][j][k][l] >= open[i][j] + open[k][l]-1 )

comm = [
    [0,   0,   1.0, 1.5, 0.0],
    [0,   0,   1.4, 1.2, 0.0],
    [1.0, 1.4, 0,   0,   2.0],
    [1.5, 1.2, 0,   0,   0.7],
    [0.0, 0,0, 2,   0.7,  0.0]
]
city = [
    [5, 14, 13],
    [14, 5, 9],
    [13, 9, 10]
]

result = 0
dept =[ [10, 15, 10, 20, 5], [10, 20, 15, 15, 15],[0,0,0,0,0]]

for i in range(3):
    for j in range(5):
        for k in range(3):
            for l in range(5):
                if  l>j:
                    result+=(path[i][j][k][l]*city[i][k]*comm[j][l])
for i in range(5):
    for j in range(3):
        result-=(dept[j][i]*open[j][i])

solver.Minimize(result)
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value()}')
    for i in range(3):
        print(f'City {i}:')
        for j in range(5):
            if open[i][j].solution_value() == 1:
                print(f'- Department {j}')
   
                
else:
    print('The solver did not find an optimal solution.')

Solution:
Objective value = -14.900000000000006
City 0:
- Department 0
- Department 3
City 1:
- Department 1
- Department 2
- Department 4
City 2:


In [ ]:
### 12.11

from ortools.linear_solver import pywraplp
import math
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()

a=solver.NumVar(-solver.infinity(),solver.infinity(),'a')
b=solver.NumVar(-solver.infinity(),solver.infinity(),'b')
result=0
x = [0.0, 0.5, 1.0, 1.5, 1.9, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.6, 7.0, 7.6, 8.5, 9.0, 10.0]
y = [1.0, 0.9, 0.7, 1.5, 2.0, 2.4, 3.2, 2.0, 2.7, 3.5, 1.0, 4.0, 3.6, 2.7, 5.7, 4.6, 6.0, 6.8, 7.3]

abs_diff = [solver.NumVar(0, solver.infinity(), f'abs_diff_{i}') for i in range(len(x))]
for i in range(len(x)):
    solver.Add(abs_diff[i] >= (a * x[i] + b - y[i]))
    solver.Add(abs_diff[i] >= -(a * x[i] + b - y[i]))

solver.Minimize(solver.Sum(abs_diff))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value()}')
    print(f'a = {a.solution_value()}')
    print(f'b = {b.solution_value()}')
else:
    print('The solver did not find an optimal solution.')

Solution:
Objective value = 11.466249999999995
a = 0.6375
b = 0.5812500000000002


In [ ]:
from ortools.linear_solver import pywraplp
import math
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()

a=solver.NumVar(-solver.infinity(),solver.infinity(),'a')
b=solver.NumVar(-solver.infinity(),solver.infinity(),'b')
res=solver.NumVar(0,solver.infinity(),'res')

for i in range(len(x)):
    solver.Add(res >= (a * x[i] + b - y[i]))
    solver.Add(res >= -(a * x[i] + b - y[i]))

solver.Minimize(res)

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value()}')
    print(f'a = {a.solution_value()}')
    print(f'b = {b.solution_value()}')
else:
    print('The solver did not find an optimal solution.')

Solution:
Objective value = 1.725
a = 0.625
b = -0.39999999999999974


In [ ]:
### 12.3

In [ ]:
dict = [
    [1, 9, 11, 34, 'A'],
    [1, 13, 47, 411, 'A'],
    [1, 14, 44, 82, 'A'],
    [1, 17, 25, 157, 'B'],
    [1, 18, 10, 5, 'A'],
    [1, 19, 26, 183, 'A'],
    [1, 23, 26, 14, 'B'],
    [1, 21, 54, 215, 'B'],
    [2, 9, 18, 102, 'B'],
    [2, 11, 51, 21, 'A'],
    [2, 17, 20, 54, 'B'],
    [2, 18, 105, 0, 'B'],
    [2, 18, 7, 6, 'B'],
    [2, 17, 16, 96, 'B'],
    [2, 22, 34, 118, 'A'],
    [2, 24, 100, 112, 'B'],
    [2, 36, 50, 535, 'B'],
    [2, 43, 21, 8, 'B'],
    [3, 6, 11, 53, 'B'],
    [3, 15, 19, 28, 'A'],
    [3, 15, 14, 69, 'B'],
    [3, 25, 10, 65, 'B'],
    [3, 39, 11, 27, 'B']
]
from ortools.linear_solver import pywraplp
import math
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()
open = [solver.BoolVar(f'M{i} belongs to D1') for i in range(23)]
deviation1 = solver.NumVar(0, solver.infinity(), 'deviation1')
deviation2 = solver.NumVar(0, solver.infinity(), 'deviation2')
deviation3 = solver.NumVar(0, solver.infinity(), 'deviation3')
deviation4 = solver.NumVar(0, solver.infinity(), 'deviation4')
deviation5 = solver.NumVar(0, solver.infinity(), 'deviation5')
deviation6 = solver.NumVar(0, solver.infinity(), 'deviation6')
deviation7 = solver.NumVar(0, solver.infinity(), 'deviation7')
temp1=0
temp2=0
temp3=0
temp4=0
temp6=temp7=0
temp8=temp9=0
t1=0
t2=0
t3=t4=0
t7=t6=0
for i in range(23):
    temp1+=dict[i][2]*open[i]
    temp2+=dict[i][2]
    temp3+=dict[i][3]*open[i]
    temp4+=dict[i][3]
    if dict[i][0]==1:
        temp6+=dict[i][1]*open[i]
        temp7+=dict[i][1]
    if dict[i][0]==2:
        temp8+=dict[i][1]*open[i]
        temp9+=dict[i][1]
    if dict[i][0]==3:
        t6+=dict[i][1]*open[i]
        t7+=dict[i][1]
    if dict[i][4]=='A':
        t1+=open[i]
        t2+=1
    if dict[i][4]=='B':
        t3+=open[i]
        t4+=1
solver.Add(temp1<=0.45*temp2)
solver.Add(temp1>=0.35*temp2)
solver.Add(temp3<=0.45*temp4)
solver.Add(temp3>=0.35*temp4)
solver.Add(temp6<=0.45*temp7)
solver.Add(temp6>=0.35*temp7)
solver.Add(t6<=0.45*t7)
solver.Add(t6>=0.35*t7)
solver.Add(temp8<=0.45*temp9)
solver.Add(temp8>=0.35*temp9)
solver.Add(t1<=0.45*t2)
solver.Add(t1>=0.35*t2)
solver.Add(t3<=0.45*t4)
solver.Add(t3>=0.35*t4)
solver.Add(deviation1>=temp1-0.4*temp2)
solver.Add(deviation1>=-temp1+0.4*temp2)
solver.Add(deviation2>=temp3-0.4*temp4)
solver.Add(deviation2>=-temp3+0.4*temp4)
solver.Add(deviation3>=temp6-0.4*temp7)
solver.Add(deviation3>=-temp6+0.4*temp7)
solver.Add(deviation4>=t6-0.4*t7)
solver.Add(deviation4>=-t6+0.4*t7)
solver.Add(deviation5>=temp8-0.4*temp9)
solver.Add(deviation5>=-temp8+0.4*temp9)
solver.Add(deviation6>=t1-0.4*t2)
solver.Add(deviation6>=-t1+0.4*t2)
solver.Add(deviation7>=t3-0.4*t4)
solver.Add(deviation7>=-t3+0.4*t4)

total_deviation = deviation1/(0.4*temp2) + deviation2/(0.4*temp4) + deviation3/(0.4*temp7) + deviation4/(0.4*t7) + deviation5/(0.4*temp9) + deviation6/(0.4*t2)+deviation7/(0.4*t4)
solver.Minimize(total_deviation)

status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    for i in range(23):
        print(f'M{i+1} belongs to D1: {open[i].solution_value()}')
else:
    print('No optimal solution found.')
    

Solution:
M1 belongs to D1: 1.0
M2 belongs to D1: 1.0
M3 belongs to D1: 1.0
M4 belongs to D1: 1.0
M5 belongs to D1: 0.0
M6 belongs to D1: 0.0
M7 belongs to D1: 0.0
M8 belongs to D1: 0.0
M9 belongs to D1: 0.0
M10 belongs to D1: 0.0
M11 belongs to D1: 1.0
M12 belongs to D1: 0.0
M13 belongs to D1: 0.0
M14 belongs to D1: 0.0
M15 belongs to D1: 0.0
M16 belongs to D1: 1.0
M17 belongs to D1: 0.0
M18 belongs to D1: 1.0
M19 belongs to D1: 0.0
M20 belongs to D1: 0.0
M21 belongs to D1: 1.0
M22 belongs to D1: 1.0
M23 belongs to D1: 0.0


In [ ]:
### 12.15

In [ ]:
import ortools.linear_solver.pywraplp as pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('Solver not created.')
    exit()

# Generator variables
ty = [[solver.IntVar(0, solver.infinity(), f'ty{i + 1} in period{j + 1}') for j in range(5)] for i in range(3)]
num = [[solver.IntVar(0, solver.infinity(), f'num{i + 1} in period{j + 1}') for j in range(5)] for i in range(3)]
start = [[solver.IntVar(0, solver.infinity(), f'start{i + 1} in period{j + 1}') for j in range(5)] for i in range(3)]

# Generator parameters
minwatt = [850, 1250, 1500]
maxwatt = [2000, 1750, 4000]
demand = [15000, 30000, 25000, 40000, 27000]
setup = [2000, 1000, 500]
avail = [12, 10, 5]
costmin = [1000, 2600, 3000]
costex = [2, 1.3, 3]

# Constraints and objective
for i in range(5):
    for j in range(3):
        solver.Add(ty[j][i] >= minwatt[j] * num[j][i])
        solver.Add(ty[j][i] <= maxwatt[j] * num[j][i])
        solver.Add(num[j][i] <= avail[j])
        if i == 0:
            solver.Add(start[j][i] == num[j][i])
        else:
            solver.Add(start[j][i] == num[j][i] - num[j][i - 1])
    solver.Add(sum(ty[j][i] for j in range(3)) >= demand[i])
    solver.Add(sum(num[j][i] * maxwatt[j] for j in range(3)) >= 1.15 * demand[i])

# Objective function
total_cost = sum(setup[j] * start[j][i] + num[j][i] * costmin[j] + (ty[j][i] - minwatt[j] * num[j][i]) * costex[j]
                for i in range(5) for j in range(3))
solver.Minimize(total_cost)

# Solve and print results
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    print('Solution:')
    for i in range(5):
        for j in range(3):
            print(f'Period {i + 1}, Ty {j + 1}:  Num = {num[j][i].solution_value(), ty[j][i].solution_value()}')
else:
    print('The problem does not have an optimal solution.')



Objective value = 256640.0
Solution:
Period 1, Ty 1:  Num = (12.0, 11500.0)
Period 1, Ty 2:  Num = (2.0, 3500.0)
Period 1, Ty 3:  Num = (0.0, 0.0)
Period 2, Ty 1:  Num = (12.0, 16000.0)
Period 2, Ty 2:  Num = (8.0, 14000.0)
Period 2, Ty 3:  Num = (0.0, 0.0)
Period 3, Ty 1:  Num = (12.0, 11000.0)
Period 3, Ty 2:  Num = (8.0, 14000.0)
Period 3, Ty 3:  Num = (0.0, 0.0)
Period 4, Ty 1:  Num = (12.0, 23000.0)
Period 4, Ty 2:  Num = (8.0, 14000.0)
Period 4, Ty 3:  Num = (2.0, 3000.0)
Period 5, Ty 1:  Num = (12.0, 10200.0)
Period 5, Ty 2:  Num = (8.0, 13800.0)
Period 5, Ty 3:  Num = (2.0, 3000.0)


In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()

ty = [[solver.NumVar(0, solver.infinity(), f'ty{i + 1} in period{j + 1}') for j in range(5)] for i in range(3)]
###open = [[solver.BoolVar(f'open{i + 1} in period{j + 1}') for j in range(5)] for i in range(3)]
num = [[solver.NumVar(0, solver.infinity(), f'num{i + 1} in period{j + 1}') for j in range(5)] for i in range(3)]
start = [[solver.NumVar(0, solver.infinity(), f'start{i + 1} in period{j + 1}') for j in range(5)] for i in range(3)]

minwatt = [850, 1250, 1500]
maxwatt = [2000, 1750, 4000]
demand = [15000, 30000, 25000, 40000, 27000]
setup = [2000, 1000, 500]
avail = [12, 10, 5]
costmin = [1000, 2600, 3000]
costex = [2, 1.3, 3]

for i in range(5):
    for j in range(3):
        solver.Add(ty[j][i]>=minwatt[j]*num[j][i])
        solver.Add(ty[j][i]<=maxwatt[j]*num[j][i])
        solver.Add(num[j][i]<=avail[j])
        if i == 0:
            solver.Add(start[j][i] == num[j][i])
        else:
            solver.Add(start[j][i] == num[j][i] - num[j][i - 1])
    solver.Add(sum(ty[j][i] for j in range(3))>=demand[i])
    solver.Add(sum(num[j][i]*maxwatt[j] for j in range(3))>=1.15*demand[i])

total_cost = 0
for i in range(5):
    for j in range(3):
        total_cost += setup[j] * start[j][i]
        total_cost += num[j][i] * costmin[j]
        total_cost +=   (ty[j][i] - minwatt[j]*num[j][i]) * costex[j]

solver.Minimize(total_cost)
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    print('Solution:')
    for i in range(5):
        for j in range(3):
            print(f'Period {i + 1}, Ty {j + 1}:')
            #print(f'  Ty = {ty[j][i].solution_value()}')
          
            print(f'  Num = {num[j][i].solution_value()}')
            #print(f'  Start = {start[j][i].solution_value()}')
else:
    print('The problem does not have an optimal solution.')


Objective value = 256354.2857142857
Solution:
Period 1, Ty 1:
  Num = 11.999999999999998
Period 1, Ty 2:
  Num = 2.7428571428571438
Period 1, Ty 3:
  Num = 0.0
Period 2, Ty 1:
  Num = 11.999999999999998
Period 2, Ty 2:
  Num = 7.8171428571428585
Period 2, Ty 3:
  Num = 0.0
Period 3, Ty 1:
  Num = 11.999999999999998
Period 3, Ty 2:
  Num = 7.8171428571428585
Period 3, Ty 3:
  Num = 0.0
Period 4, Ty 1:
  Num = 11.999999999999998
Period 4, Ty 2:
  Num = 7.8171428571428585
Period 4, Ty 3:
  Num = 2.0800000000000005
Period 5, Ty 1:
  Num = 11.999999999999998
Period 5, Ty 2:
  Num = 7.8171428571428585
Period 5, Ty 3:
  Num = 2.080000000000001


In [ ]:
### 12.16

In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()

facttodepot=[[solver.IntVar(0,solver.infinity(),f'factory{i} to depot{j}') for j in range(4)] for i in range(2)]
facttocust=[[solver.IntVar(0,solver.infinity(),f'factory{i} to cust{j}') for j in range(6)] for i in range(2)]
depottocust=[[solver.IntVar(0,solver.infinity(),f'depot{i} to cust{j}') for j in range(6)] for i in range(4)]
f_c=[150000,200000]
d_c=[70000,50000,100000,40000]
demand=[50000,10000,40000,35000,60000,20000]


### have 0 possiblility 
solver.Add(facttodepot[1][0]==0)
solver.Add(facttocust[0][1]==0)
solver.Add(facttocust[0][4]==0)
for i in range(1,6):
    solver.Add(facttocust[1][i]==0)
solver.Add(depottocust[0][0]==0)
solver.Add(depottocust[0][4]==0)
solver.Add(depottocust[1][5]==0)
solver.Add(depottocust[2][0]==0)
solver.Add(depottocust[2][3]==0)
solver.Add(depottocust[3][0]==0)
solver.Add(depottocust[3][1]==0)

###   capacity of fcatory check 
for i in range(2):
    temp=0
    for j in range(4):
        temp+=facttodepot[i][j]
    for j in range(6):
        temp+=facttocust[i][j]
    solver.Add(temp<=f_c[i])

###  depot check 
for i in range(4):
    solver.Add(sum(facttodepot[j][i] for j in range(2))>=sum(depottocust[i][j] for j in range(6)))
    solver.Add(sum(facttodepot[j][i] for j in range(2))<=d_c[i])

## customer check 
for i in range(6):
    solver.Add(sum(depottocust[j][i] for j in range(4))+sum(facttocust[j][i] for j in range(2))>=demand[i])
    if i==0:
        for j in range(2):
            solver.Add(facttocust[0][0]>=facttocust[j][i])
        for j in range(4):
            solver.Add(facttocust[0][0]>=depottocust[j][i])
    if i==1:
        for j in range(2):
            solver.Add(depottocust[0][1]>=facttocust[j][i])
        for j in range(4):
            solver.Add(depottocust[0][1]>=depottocust[j][i])
    if i==4:
        for j in range(2):
            solver.Add(depottocust[1][4]>=facttocust[j][i])
        for j in range(4):
            solver.Add(depottocust[1][4]>=depottocust[j][i])
    if i==5:
        for j in range(2):
            solver.Add(depottocust[2][5]>=facttocust[j][i])
            solver.Add(depottocust[3][i]>=facttocust[j][i])
        for j in range(4):
            solver.Add(depottocust[2][5]>=depottocust[j][i])
            solver.Add(depottocust[3][i]>=depottocust[j][i])
result=0
dat1=[
    [0.5, 0.0],  
    [0.5, 0.3],  
    [1.0, 0.5],  
    [0.2, 0.2]   
]
for i in range(2):
    for j in range(4):
        result+=facttodepot[i][j]*dat1[j][i]
        result+=0
dat2=[[1,0,1.5,2,0,1],[2,0,0,0,0,0]]
for i in range(2):
    for j in range(6):
        result+=facttocust[i][j]*dat2[i][j]
dat3=[[0,1.5,0.5,1.5,0,1],[1,0.5,0.5,1,0.5,0],[0,1.5,2,0,0.5,1.5],[0,0,0.2,1.5,0.5,1.5]]
for i in range(4):
    for j in range(6):
        result+=depottocust[i][j]*dat3[i][j]

solver.Minimize(result)
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print()
    for i in range(2):
        for j in range(4):
            print(f'factory{i} to depot{j} =', facttodepot[i][j].solution_value())
    print()
    for i in range(2):
        for j in range(6):
            print(f'factory{i} to cust{j} =', facttocust[i][j].solution_value())
    print()
    for i in range(4):
        for j in range(6):
            print(f'depot{i} to cust{j} =', depottocust[i][j].solution_value())
else:
    print('The problem does not have an optimal solution.')




Solution:
Objective value = 227500.0

factory0 to depot0 = 35000.0
factory0 to depot1 = 0.0
factory0 to depot2 = 0.0
factory0 to depot3 = 0.0
factory1 to depot0 = 0.0
factory1 to depot1 = 50000.0
factory1 to depot2 = 35000.0
factory1 to depot3 = 40000.0

factory0 to cust0 = 50000.0
factory0 to cust1 = 0.0
factory0 to cust2 = 0.0
factory0 to cust3 = 0.0
factory0 to cust4 = 0.0
factory0 to cust5 = 5000.0
factory1 to cust0 = 0.0
factory1 to cust1 = 0.0
factory1 to cust2 = 0.0
factory1 to cust3 = 0.0
factory1 to cust4 = 0.0
factory1 to cust5 = 0.0

depot0 to cust0 = 0.0
depot0 to cust1 = 5000.0
depot0 to cust2 = 5000.0
depot0 to cust3 = 20000.0
depot0 to cust4 = 0.0
depot0 to cust5 = 5000.0
depot1 to cust0 = 0.0
depot1 to cust1 = 5000.0
depot1 to cust2 = 0.0
depot1 to cust3 = 15000.0
depot1 to cust4 = 30000.0
depot1 to cust5 = 0.0
depot2 to cust0 = 0.0
depot2 to cust1 = 0.0
depot2 to cust2 = 0.0
depot2 to cust3 = 0.0
depot2 to cust4 = 30000.0
depot2 to cust5 = 5000.0
depot3 to cust0 = 0.0


In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()
lines=[solver.BoolVar(f'line {i} has all same ') for i in range(49)]
cell=[solver.BoolVar(f'cell layer{i} is black') for i in range(27)] 
cells_line = [
    [1, 2, 3], [4, 5, 6], [7, 8, 9],
    [10, 11, 12], [13, 14, 15], [16, 17, 18],
    [19, 20, 21], [22, 23, 24], [25, 26, 27],
    [1, 4, 7], [2, 5, 8], [3,6, 9],
    [10, 13, 16], [11, 14, 17], [12, 15, 18],
    [19, 22, 25], [20, 23, 26], [21, 24, 27],
    [1, 5, 9], [3, 5, 7], [10, 14, 18], [16, 14, 12], [19, 13, 17], [21, 23, 25],
    [1, 10, 19], [2, 11, 20], [3, 12, 21], [1, 11, 21], [3, 11, 19],
    [4, 13, 22], [5, 14, 23], [6, 15, 24], [4, 14, 24], [6, 14, 22],
    [7, 16, 25], [8, 17, 26], [9, 18, 27], [7, 17, 27], [9, 17, 25],
    [1, 13, 25], [7, 13, 19], [3, 15, 27], [21, 15, 9],
    [1, 14, 27], [3, 14, 25], [7, 14, 21], [9, 14, 19], [2, 14, 26], [8, 14, 20]
]

solver.Add(sum(cell[i] for i in range(27))==14)
for i in range(49):
    solver.Add(cell[cells_line[i][0]-1]+cell[cells_line[i][1]-1]+cell[cells_line[i][2]-1]-lines[i]<=2)
    solver.Add(cell[cells_line[i][0]-1]+cell[cells_line[i][1]-1]+cell[cells_line[i][2]-1]+lines[i]>=1)
solver.Minimize(sum(lines[i] for i in range(49)))
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())

Solution:
Objective value = 2.9999999999999996


In [ ]:
from ortools.linear_solver import pywraplp

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()

# Define variables
lines = [solver.BoolVar(f'line {i} has all same') for i in range(49)]
cell = [solver.BoolVar(f'cell layer{i} is black') for i in range(27)]

# Define cell lines
cells_line = [
    [0, 1, 2], [3, 4, 5], [6, 7, 8],
    [9, 10, 11], [12, 13, 14], [15, 16, 17],
    [18, 19, 20], [21, 22, 23], [24, 25, 26],
    [0, 3, 6], [1, 4, 7], [2, 5, 8],
    [9, 12, 15], [10, 13, 16], [11, 14, 17],
    [18, 21, 24], [19, 22, 25], [20, 23, 26],
    [0, 4, 8], [2, 4, 6], [9, 13, 17], [15, 13, 11], [18, 22, 26], [20, 22, 24],
    [0, 9, 18], [1, 10, 19], [2, 11, 20], [0, 10, 20], [2, 10, 18],
    [3, 12, 21], [4, 13, 22], [5, 14, 23], [3, 13, 23], [5, 13, 21],
    [6, 15, 24], [7, 16, 25], [8, 17, 26], [6, 16, 26], [8, 16, 24],
    [0, 12, 24], [6, 12, 18], [2, 14, 26], [20, 14, 8],
    [0, 13, 26], [2, 13, 24], [6, 13, 20], [8, 13, 18], [1, 13, 25], [7, 13, 19]
]


# Add constraints
solver.Add(sum(cell[i] for i in range(27)) == 14)
for i in range(49):
    solver.Add(cell[cells_line[i][0]] + cell[cells_line[i][1]] + cell[cells_line[i][2]] - lines[i] <= 2)
    solver.Add(cell[cells_line[i][0]] + cell[cells_line[i][1]] + cell[cells_line[i][2]] + lines[i] >= 1)

# Define the objective function
solver.Minimize(sum(lines[i] for i in range(49)))

# Solve the problem
status = solver.Solve()

# Check if a solution is found
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', round(solver.Objective().Value()))
    
    # Print the values of the decision variables
    for i in range(27):
        print(f'cell {i} is black:', round(cell[i].solution_value()))
    
    for i in range(49):
        print(f'line {i} has all same:', round(lines[i].solution_value()))
else:
    print('The problem does not have an optimal solution.')


Solution:
Objective value = 4
cell 0 is black: 0
cell 1 is black: 0
cell 2 is black: 1
cell 3 is black: 0
cell 4 is black: 1
cell 5 is black: 0
cell 6 is black: 1
cell 7 is black: 0
cell 8 is black: 1
cell 9 is black: 1
cell 10 is black: 1
cell 11 is black: 0
cell 12 is black: 1
cell 13 is black: 0
cell 14 is black: 0
cell 15 is black: 0
cell 16 is black: 0
cell 17 is black: 1
cell 18 is black: 0
cell 19 is black: 1
cell 20 is black: 1
cell 21 is black: 1
cell 22 is black: 0
cell 23 is black: 1
cell 24 is black: 0
cell 25 is black: 1
cell 26 is black: 1
line 0 has all same: 0
line 1 has all same: 0
line 2 has all same: 0
line 3 has all same: 0
line 4 has all same: 0
line 5 has all same: 0
line 6 has all same: 0
line 7 has all same: 0
line 8 has all same: 0
line 9 has all same: 0
line 10 has all same: 0
line 11 has all same: 0
line 12 has all same: 0
line 13 has all same: 0
line 14 has all same: 0
line 15 has all same: 0
line 16 has all same: 0
line 17 has all same: 1
line 18 has all sa

In [ ]:
####   12.23

Solution:
Objective value = 108.48153723234383


Solution:
Objective value = 2.0
Route for vehicle 0:
[(0, 11), (2, 9), (3, 13), (4, 10), (6, 5), (7, 14), (11, 9), (12, 9), (14, 4)]
Route for vehicle 2:
[(0, 1), (1, 0), (5, 6), (8, 14), (9, 12), (10, 11), (13, 0)]


In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('Solver not created.')
    exit()

blocks = [[0], [1, 2, 3, 4], [5, 6, 7, 8, 9, 10, 11, 12, 13], [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]]
value=[6,5,4,12,6,2,2,0.5,3,3,1,4,4,2,0.75,0.75,0.5,0.25,1,1,0.75,0.5,1.5,2,1.5,0.75,1.5,1.5,1.5,0.75]
remove=[solver.BoolVar(f'block{i} is removed') for i in range(30)]

above={0: [1, 2, 3, 4], 1: [5, 6, 8, 9], 2: [6, 7, 9, 10], 3: [8, 9, 11, 12], 4: [12, 13, 9, 10], 5: [14, 15, 18, 19], 6: [15, 16, 20, 19], 7: [16, 17, 20, 21], 8: [18, 19, 22, 23], 9: [19, 20, 23, 24], 10: [20, 21, 24, 25], 11: [22, 23, 26, 27], 12: [23, 24, 27, 28], 13: [24, 25, 28, 29]} 
for i in above.keys():
    for j in above[i]:
        solver.Add(remove[j]>=remove[i])

result=0
for i in range(4):
    if i==0:
        for j in range(14,30):
            result+=remove[j]*(value[j]*2000-3000)
    if i==1:
        for j in range(5,14):
            result+=remove[j]*(value[j]*2000-6000)
    if i==2:
        for j in range(1,5):
            result+=remove[j]*(value[j]*2000-8000)
    if i==3:
        for j in range(1):
            result+=remove[j]*(value[j]*2000-10000)
solver.Maximize(result)
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())          
    


Solution:
Objective value = 17500.0


In [ ]:
from ortools.linear_solver import pywraplp
from itertools import combinations

solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('Solver not created.')
    exit()

coff = [23, 21, 19, 17, 14, 13, 13, 9]
rhs = 70
a = [solver.IntVar(0, solver.infinity(), f'value of {i} coff') for i in range(8)]
arhs = solver.IntVar(0, solver.infinity(), 'arhs')

for i in range(7):
    solver.Add(a[i] >= a[i + 1])

ceilings = []
roofs = []

for i in range(8,-1,-1):
    combs = combinations(range(8), i)
    for j in combs:
        res = sum(coff[k] for k in j)
        if res <= rhs:
            ceilings.append(j)
        if res >= rhs + 1:
            roofs.append(j)


for i in ceilings:
    solver.Add(sum(a[j] for j in i) <= arhs)

for i in roofs:
    solver.Add(sum(a[j] for j in i) >= arhs + 1)

result = arhs - a[2] - a[5]
solver.Minimize(result)

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value()) 
    for i in range(8):
        print(f'a[{i}] =', a[i].solution_value())
    print('arhs =', arhs.solution_value())
else:
    print('The problem does not have an optimal solution.')


Solution:
Objective value = 26.000000000000007
a[0] = 16.0
a[1] = 14.0
a[2] = 13.0
a[3] = 12.0
a[4] = 10.0
a[5] = 9.0
a[6] = 9.0
a[7] = 6.0
arhs = 48.0


In [3]:
from ortools.linear_solver import pywraplp
import math

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Solver not created.')
    exit()

num_farms = 21
num_days = 2

x = [[[solver.BoolVar(f'GOES from {i} to {j} in day {k}') for k in range(num_days)] for j in range(num_farms)] for i in range(num_farms)]
y = [[solver.BoolVar(f'goes to city{i} in day {k}') for k in range(num_days)] for i in range(num_farms)]
u = [[solver.NumVar(0, 21, f'position of city{i} in day {k}') for k in range(num_days)] for i in range(num_farms)]

farms_data = {
    0: {"Position": (0, 0), "Collection Frequency": None, "Requirement": 0},
    1: {"Position": (-3, 3), "Collection Frequency": "Every day", "Requirement": 5},
    2: {"Position": (1, 11), "Collection Frequency": "Every day", "Requirement": 4},
    3: {"Position": (4, 7), "Collection Frequency": "Every day", "Requirement": 3},
    4: {"Position": (-5, 9), "Collection Frequency": "Every day", "Requirement": 6},
    5: {"Position": (-5, -2), "Collection Frequency": "Every day", "Requirement": 7},
    6: {"Position": (-4, -7), "Collection Frequency": "Every day", "Requirement": 3},
    7: {"Position": (6, 0), "Collection Frequency": "Every day", "Requirement": 4},
    8: {"Position": (3, -6), "Collection Frequency": "Every day", "Requirement": 6},
    9: {"Position": (-1, -3), "Collection Frequency": "Every day", "Requirement": 5},
    10: {"Position": (0, -6), "Collection Frequency": "Every other day", "Requirement": 4},
    11: {"Position": (6, 4), "Collection Frequency": "Every other day", "Requirement": 7},
    12: {"Position": (2, 5), "Collection Frequency": "Every other day", "Requirement": 3},
    13: {"Position": (-2, 8), "Collection Frequency": "Every other day", "Requirement": 4},
    14: {"Position": (6, 10), "Collection Frequency": "Every other day", "Requirement": 5},
    15: {"Position": (1, 8), "Collection Frequency": "Every other day", "Requirement": 6},
    16: {"Position": (-3, 1), "Collection Frequency": "Every other day", "Requirement": 8},
    17: {"Position": (-6, 5), "Collection Frequency": "Every other day", "Requirement": 5},
    18: {"Position": (2, 9), "Collection Frequency": "Every other day", "Requirement": 7},
    19: {"Position": (-6, -5), "Collection Frequency": "Every other day", "Requirement": 6},
    20: {"Position": (5, -4), "Collection Frequency": "Every other day", "Requirement": 6},
}

def distance(i, j):
    return math.sqrt((farms_data[i]["Position"][0] - farms_data[j]["Position"][0])**2 +
                     (farms_data[i]["Position"][1] - farms_data[j]["Position"][1])**2)

for k in range(num_days):
    for i in range(21):
        solver.Add(x[i][i][k]==0)

        if i < 10:
            solver.Add(y[i][k] == 1)  # Ensure "everyday" farms are visited
        else:
            solver.Add(y[i][0]+y[i][1]==  1)  # "Every other day" farms can be visited at most once per day
        solver.Add(sum(x[i][j][k] for j in range(21)) == y[i][k])  # Outgoing edges constraint
        solver.Add(sum(x[j][i][k] for j in range(21)) == y[i][k])  # Incoming edges constraint

    for i in range(21):
        for j in range(21):
            solver.Add(x[i][j][k] + x[j][i][k] <= 1)  # Ensure no subtours
    for i in range( 1,21):
        for j in range(1, 21):
            if i != j:
                solver.Add(u[i][k] - u[j][k] + 21 * x[i][j][k] <= 20)
    solver.Add(u[0][k]==0)

    # Capacity constraint for each day
    solver.Add(sum(y[i][k] * farms_data[i]["Requirement"] for i in range(21)) <= 80)

# Objective function: minimize total distance
total_distance = 0
for k in range(num_days):
    for i in range(21):
        for j in range(21):
            total_distance += distance(i, j) * x[i][j][k]

solver.Minimize(total_distance)

# Solve the problem
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    
    for k in range(num_days):
        print(f"for day {k}")
        start = 0
        path = []
        while True:
            path.append(start+1)
            next_found = False
            for j in range(21):
                if x[start][j][k].solution_value() >= 0.5:
                    start = j
                    next_found = True
                    break
            if not next_found or start == 0:
                break
        path.append(1)
        print(path)
        


Solution:
Objective value = 123.05623785349735
for day 0
[1, 10, 6, 20, 7, 11, 9, 21, 8, 13, 4, 15, 3, 5, 18, 2, 17, 1]
for day 1
[1, 10, 6, 7, 9, 8, 12, 4, 16, 19, 3, 14, 5, 2, 1]


In [1]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("demand_elasticity")

milk = m.addVar(lb=0, name="milk")
butter = m.addVar(lb=0, name="butter")
cheese1 = m.addVar(lb=0, name="cheese1")
cheese2 = m.addVar(lb=0, name="cheese2")

prices = [297, 720, 1050, 815]
m.addConstr(0.04 * milk + 0.80 * butter + 0.35 * cheese1 + 0.25 * cheese2 <= 600000)
m.addConstr(0.09 * milk + 0.02 * butter + 0.3 * cheese1 + 0.4 * cheese2 <= 750000)

demand = [4820000, 320000, 210000, 70000]
e = [0.4, 2.7, 1.1, 0.4, 0.1, 0.4]

new_price = [m.addVar(lb=0, name=f"prices{i}") for i in range(4)]
new_demand = [milk, butter, cheese1, cheese2]

for i in range(4):
    m.addConstr(1 - (new_demand[i] / demand[i]) == e[i] * ((new_price[i] / prices[i]) - 1))

m.addConstr(1 - (new_demand[2] / demand[2]) == e[4] * (1 - (new_price[3] / prices[3])))
m.addConstr(1 - (new_demand[3] / demand[3]) == e[5] * (1 - (new_price[2] / prices[2])))

temp1 = gp.LinExpr()
temp2 = gp.LinExpr()
for i in range(4):
    temp1 += demand[i] * new_price[i]
    temp2 += demand[i] * prices[i]
m.addConstr(temp1 <= temp2)

result = gp.LinExpr()
for i in range(4):
    result += new_price[i] * ((1 - e[i] * ((new_price[i] / prices[i]) - 1)) * demand[i])

m.setObjective(result, GRB.MAXIMIZE)

m.optimize()

if m.status == GRB.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {m.objVal}')
    for var in new_price:
        print(f'{var.varName} = {var.x}')
else:
    print('No optimal solution found')


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 8 columns and 24 nonzeros
Model fingerprint: 0x234e86a2
Model has 4 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-07, 5e+06]
  Objective range  [1e+05, 7e+06]
  QObjective range [7e+01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 2e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 4 rows and 3 columns
Presolve time: 0.02s
Presolved: 5 rows, 5 columns, 14 nonzeros
Presolved model has 4 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 8.000e+00
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+0

In [40]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('Solver not created.')
    exit()

# Distance matrix and problem parameters
distance_matrix = [
    [0, 20, 25, 35, 65, 90, 85, 80, 86, 25, 35, 20, 44, 35, 82],  # Heathrow
    [20, 0, 15, 35, 60, 55, 57, 85, 90, 25, 35, 30, 37, 20, 40],  # Harrow
    [25, 15, 0, 30, 50, 70, 55, 50, 65, 10, 25, 15, 24, 20, 90],  # Ealing
    [35, 35, 30, 0, 45, 60, 53, 55, 47, 12, 22, 20, 12, 10, 21],  # Holborn
    [65, 60, 50, 45, 0, 46, 15, 45, 75, 25, 11, 19, 15, 25, 25],  # Sutton
    [90, 55, 70, 60, 46, 0, 15, 25, 45, 65, 53, 43, 63, 70, 27],  # Dartford
    [85, 57, 55, 53, 15, 15, 0, 17, 25, 41, 25, 33, 27, 45, 30],  # Bromley
    [80, 85, 50, 55, 45, 25, 17, 0, 25, 40, 34, 32, 20, 30, 10],  # Greenwich
    [86, 90, 65, 47, 75, 45, 25, 25, 0, 65, 70, 72, 61, 45, 13],  # Barking
    [25, 25, 10, 12, 25, 65, 41, 40, 65, 0, 20, 8, 7, 15, 25],    # Hammersmith
    [35, 35, 25, 22, 11, 53, 25, 34, 70, 20, 0, 5, 12, 45, 65],   # Kingston
    [20, 30, 15, 20, 19, 43, 33, 32, 72, 8, 5, 0, 14, 34, 56],    # Richmond
    [44, 37, 24, 12, 15, 63, 27, 20, 61, 7, 12, 14, 0, 30, 40],   # Battersea
    [35, 20, 20, 10, 25, 70, 45, 30, 45, 15, 45, 34, 30, 0, 27],  # Islington
    [82, 40, 90, 21, 25, 27, 30, 10, 13, 25, 65, 56, 40, 27, 0]   # Woolwich
]

num_cities = len(distance_matrix)
num_vehicles = 6

# Decision variables
used = [solver.BoolVar(f'vehicle_{i}_used') for i in range(num_vehicles)]
path = [[[solver.BoolVar(f'vehicle_{k}_from_{i}_to_{j}') for j in range(num_cities)] for i in range(num_cities)] for k in range(num_vehicles)]
visit = [[solver.BoolVar(f'vehicle_{k}_visits_city_{i}') for i in range(num_cities)] for k in range(num_vehicles)]
u = [[solver.IntVar(0,num_cities, f'position_of_{i}_in_vehicle_{k}') for i in range(num_cities)] for k in range(num_vehicles)]

# Constraints
for k in range(num_vehicles):
    for i in range(num_cities):
        solver.Add(path[k][i][i] == 0)  # Ensure no self-loops
        solver.Add(visit[k][i] <= used[k])  # Vehicle k visits city j only if used[k] is 1
        solver.Add(sum(path[k][j][i] for j in range(num_cities)) == visit[k][i])
        solver.Add(sum(path[k][i][j] for j in range(num_cities)) == visit[k][i])  # Ensure exactly one inbound and one outbound edge for each city

    # Capacity constraint for vehicle k
    solver.Add(sum(distance_matrix[i][j] * path[k][i][j] for i in range(num_cities) for j in range(1,num_cities)) <= 120)
    
# Ensure each city is visited exactly once by one vehicle
for i in range(num_cities):
    if i == 0:
        for k in range(num_vehicles):
            solver.Add(visit[k][i] == used[k])
    else:
        solver.Add(sum(visit[k][i] for k in range(num_vehicles)) == 1)

# Ensure each vehicle starts from Heathrow
for k in range(num_vehicles):
    solver.Add(sum(path[k][0][j] for j in range(1, num_cities)) == used[k])
    solver.Add(sum(path[k][j][0] for j in range(1, num_cities)) == used[k])

# Subtour elimination constraints
for k in range(num_vehicles):
    for i in range(1, num_cities):
        for j in range(1, num_cities):
            if i != j:
                solver.Add(u[k][i] - u[k][j] + num_cities * path[k][i][j] <= num_cities - 1)

for k in range(num_vehicles):
    for i in range(num_cities):
        solver.Add(u[k][i] >= visit[k][i])
        solver.Add(u[k][i] <= num_cities * visit[k][i])
for i in range(6):
    for j in range(6):
        if i<=j:
            solver.Add(sum(visit[i][k] for k in range(num_cities))>=sum(visit[j][k] for k in range(num_cities)))

# Objective function
solver.Minimize(sum(used[k] for k in range(num_vehicles)))

# Solve and print solution
status = solver.Solve()


if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for k in range(num_vehicles):
        if used[k].solution_value() == 1:
            print(f'Route for vehicle {k}:')
            route = []
            current_city = 0
            while True:
                route.append(current_city)
                next_city = None
                for j in range(num_cities):
                    if path[k][current_city][j].solution_value() > 0.5:
                        next_city = j
                        break
                if next_city is None or next_city == 0:
                    break
                current_city = next_city
            print(route)
            total_distance = sum(distance_matrix[i][j] * path[k][i][j].solution_value() for i in range(num_cities) for j in range(1,num_cities))
            print(f'Total distance: {total_distance}')
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 2.0
Route for vehicle 0:
[0, 1, 2, 9, 3, 12, 6, 5]
Total distance: 111.0
Route for vehicle 1:
[0, 11, 10, 4, 13, 7, 14, 8]
Total distance: 114.0


In [10]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('Solver not created.')
    exit()
cities_data = {
    0: {
        'City': 'Winchester',
        'Values': [7, 8, 10, 12, 8.5, 4, 2, 0.6, 1.5]
    },
    1: {
        'City': 'Andover',
        'Values': [6, 6, 20, 30, 9, 4.5, 2.3, 0.7, 1.6]
    },
    2: {
        'City': 'Basingstoke',
        'Values': [2, 3, 40, 40, 2, 1.5, 0.8, 0.25, 0.5]
    },
    3: {
        'City': 'Poole',
        'Values': [14, 9, 20, 25, 10, 6, 2.6, 0.86, 1.9]
    },
    4: {
        'City': 'Woking',
        'Values': [10, 9, 10, 10, 11, 5, 2.4, 1, 2]
    },
    5: {
        'City': 'Newbury',
        'Values': [24, 15, 15, 13, 25, 1.9, 8, 2.6, 4.5]
    },
    6: {
        'City': 'Portsmouth',
        'Values': [6, 7, 50, 40, 8.5, 3, 2.5, 0.9, 1.6]
    },
    7: {
        'City': 'Alresford',
        'Values': [8, 7.5, 5, 8, 9, 4, 2.1, 0.85, 2]
    },
    8: {
        'City': 'Salisbury',
        'Values': [5, 5, 10, 10, 5, 2.5, 2, 0.65, 0.9]
    },
    9: {
        'City': 'Guildford',
        'Values': [8, 10, 30, 35, 9.5, 4.5, 2.05, 0.75, 1.7]
    },
    10: {
        'City': 'Alton',
        'Values': [7, 8, 7, 8, 3, 2, 1.9, 0.70, 0.5]
    },
    11: {
        'City': 'Weybridge',
        'Values': [5, 6.5, 9, 12, 8, 4.5, 1.8, 0.63, 1.4]
    },
    12: {
        'City': 'Dorchester',
        'Values': [6, 7.5, 10, 10, 7.5, 4, 1.5, 0.45, 1.45]
    },
    13: {
        'City': 'Bridport',
        'Values': [11, 8, 8, 10, 10, 6, 2.2, 0.65, 2.2]
    },
    14: {
        'City': 'Weymouth',
        'Values': [4, 5, 10, 10, 7.5, 3.5, 1.8, 0.62, 1.6]
    },
    15: {
        'City': 'Portland',
        'Values': [3, 3.5, 3, 20, 2, 1.5, 0.9, 0.35, 0.5]
    },
    16: {
        'City': 'Chichester',
        'Values': [5, 5.5, 8, 10, 7, 3.5, 1.2, 0.45, 1.3]
    },
    17: {
        'City': 'Petersfield',
        'Values': [21, 12, 6, 6, 15, 8, 6, 0.25, 2.9]
    },
    18: {
        'City': 'Petworth',
        'Values': [6, 5.5, 2, 2, 8, 5, 1.5, 0.55, 1.55]
    },
    19: {
        'City': 'Midhurst',
        'Values': [3, 3.6, 3, 3, 2.5, 1.5, 0.8, 0.20, 0.45]
    },
    20: {
        'City': 'Reading',
        'Values': [30, 29, 120, 80, 35, 20, 7, 2.5, 8]
    },
    21: {
        'City': 'Southampton',
        'Values': [25, 16, 110, 80, 27, 12, 6.5, 3.5, 5.4]
    },
    22: {
        'City': 'Bournemouth',
        'Values': [19, 10, 90, 22, 25, 13, 5.5, 3.1, 4.5]
    },
    23: {
        'City': 'Henley',
        'Values': [7, 6, 5, 7, 8.5, 4.5, 1.2, 0.48, 2]
    },
    24: {
        'City': 'Maidenhead',
        'Values': [12, 8, 7, 10, 12, 7, 4.5, 2, 2.3]
    },
    25: {
        'City': 'Fareham',
        'Values': [4, 6, 1, 1, 7.5, 3.5, 1.1, 0.48, 1.7]
    },
    26: {
        'City': 'Romsey',
        'Values': [2, 2.5, 1, 1, 2.5, 1, 0.4, 0.1, 0.55]
    },
    27: {
        'City': 'Ringwood',
        'Values': [2, 3.5, 2, 2, 1.9, 1.2, 0.3, 0.09, 0.4]
    }
}
prop=[solver.NumVar(0,solver.infinity(),f'garage{i} has these many garage') for i in range(28) ]
w=sum(prop[i] for i in range(28))

for i in range(6):
    for j in range(28):
        temp1=0
        for k in range(28):
            temp1+=prop[k]*cities_data[k]['Values'][i]
        solver.Add(temp1<=cities_data[j]['Values'][i])
for i in range(6,9):
    for j in range(28):
        temp1=0
        
        for k in range(28):
            temp1+=prop[k]*cities_data[k]['Values'][i]
            temp2=w*cities_data[j]['Values'][i]
        solver.Add(temp1>=temp2)

solver.Maximize(w)
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
 
    for i in range(28):
        print(prop[i].solution_value())
       
else:
    print("None")


Solution:
Objective value = 0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('Solver not created.')
    exit()


price=[
        [[1200, 1000, 950], [1400, 1300, 1150], [1500, 900, 850]],  # First class
        [[900, 800, 600], [1100, 900, 750], [820, 800, 500]],  # Business class
        [[500, 300, 200], [700, 400, 350], [480, 470, 450]]  # Economy class
    ]  ### prices[class][period][option]
prob=[0.1,0.7,0.2] ### scenario
demand=[[
    [
        [[10, 15, 20], [20, 25, 35], [45, 50, 60]],  # Period 1
        [[20, 25, 35], [10, 40, 50], [50, 55, 80]],  # Period 2
        [[30, 35, 40], [30, 40, 60], [50, 70, 80]]   # Period 3
    ]
],[
    [
        [[20, 25, 35], [42, 45, 46], [45, 46, 47]],  # Period 1
        [[40, 42, 45], [50, 60, 80], [20, 30, 50]],  # Period 2
        [[40, 50, 55], [10, 40, 45], [40, 45, 60]]   # Period 3
    ]
],[
    [
        [[45, 55, 60], [50, 52, 63], [55, 56, 64]],  # Period 1
        [[50, 52, 60], [60, 65, 90], [10, 40, 60]],  # Period 2
        [[50, 60, 80], [50, 60, 70], [60, 65, 70]]   # Period 3
    ]
]
    
]  ### demand[class][period][scenario][option]


price_check=[[[solver.BoolVar(f'class {i} period{j} option{k}') for k in range(3)]for j in range(3)]for i in range(3)]
sold=[[[[solver.IntVar(0,solver.infinity(),f'class{i} period{j} scenario{k} option{l}') for l in range(3)]  for k in range(3)] for j in range(3)] for i in range(3)]
slack=[[[[solver.IntVar(0,solver.infinity(),f'class{i} period{j} scenario{k} option{l}') for l in range(3)]  for k in range(3)] for j in range(3)] for i in range(3)]


